#### Importing Library

In [77]:
import logging
import win32com.client
import sqlalchemy
import os
import pandas as pd
import pyodbc as py
import warnings
warnings.filterwarnings('ignore')

#### Config Variable

In [78]:
conn_str = (
    r'DRIVER={SQL Server};'
    r'SERVER=UG-BLR-L262;'
    r'database =AdventureWorksDW2008R2;'
    r'username=ETLdemo;'
    r'password=Devraj@@1;'
    r'Trusted_Connection=yes;'
)

#### Email body

In [79]:
body = r'''
<html>
<head>
<img src="C:\Users\prakash.raj\Downloads\header.jpg" width="900" height="100">
<title></title></head>
<body background color="white">
<h3>Email contains log files for AB testing<\h3>
</body>
</html>'''

#### Email Block

In [80]:
def sendmail():
    from_addr = 'prakash.raj@ugamsolutions.com'
    to_addr = 'prakash.raj@ugamsolutions.com'
    log_file_path = 'C:\\Users\\prakash.raj\\AB testing\\demo1.log'
    outlook = win32com.client.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = to_addr
    mail.Subject = 'AB testing log'
    mail.HTMLBody = body
    mail.Attachments.Add('C:\\Users\\prakash.raj\\AB testing\\demo1.log')
    mail.Send()

#### Logging Config

In [85]:
logging.basicConfig(filename='demo1.log' ,filemode='w',level=logging.error,format='%(asctime)s %(levelname)s-%(message)s')

#### Extract function

In [86]:
def extract():
    
        #Connection being established
    try:
        cnxn=py.connect(conn_str)
        logging.info("Connection created")
        #query being executed
        sql = """Select CustomerKey,FirstName,LastName,BirthDate,Phone from AdventureWorksDW2008R2.dbo.DimCustomer"""
        results = pd.read_sql(sql, cnxn)
        logging.info('Query Extracted sucessfully')
        #reading excel file
        df=pd.read_excel("C:/Users/prakash.raj/Downloads/sampledatafoodsales/sampledatafoodsales.xlsx",sheet_name="FoodSales")
        logging.info('excel extracted sucessfully')
        #reading json file
        df1=pd.read_json("C:/Users/prakash.raj/Downloads/sampledatafoodsales/food.json")
        logging.info('json extracted sucessfully')
        #transforamtion in process
        df.set_index("CustomerKey", inplace=True)
        df1.set_index("Category", inplace=True)
        df2=results.join(df, on="CustomerKey", how="inner")
        df3=df2.join(df1, on="Category", how="inner")
        df3.set_index("order_id", inplace=True)
        load(df3)
    except:
        logging.exception("data transformation block")
    finally:
        cnxn.close()
   

#### Load Function

In [87]:
def load(df):
    try:
        tbl='sales_detail'
        logging.info("calling load")
        server = 'UG-BLR-L262'
        database = 'ETLtest'
        #creating engine
        sqlcon = sqlalchemy.create_engine('mssql+pyodbc://@' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')
        sqlcon.connect()
        logging.info("engine created")
        df.to_sql(tbl,schema='dbo',con=sqlcon, if_exists ='replace')
        logging.info("table loaded sucessfully")
        #loading to dashboard
        df.to_excel("C:\\Users\\prakash.raj\\Documents\\Dashboard.xlsx", sheet_name='raw')
        logging.info("Excel loaded sucessfully")
    except:
        logging.exception("failed in loading block")     

#### Main

In [88]:
try:
    extract()
    #sendmail()
except NameError as e1:
    logging.exception(e1)
except SyntaxError as e2:
    logging.exception(e2)
except IndentationError as e3:
    logging.exception(e3)
except:
    print()
finally:
    logging.shutdown() 

In [ ]:
pip install streamlit-authenticator

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Created wheel for streamlit-authenticator: filename=streamlit_authenticator-0.2.1-py3-none-any.whl size=10199 sha256=f59ea2131e427d03cf170b7c66676acb19be1698f77f0db87e4326c26770be16
  Stored in directory: c:\users\prakash.raj\appdata\local\pip\cache\wheels\8f\8d\7a\2e7ee6bacf17de9d9d855ef1ee502af89b2c2bc0e24f7393f7
Successfully built streamlit-authenticator
